In [87]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras import models, datasets, utils
import os
import numpy as np 
from matplotlib import pyplot as plt

In [88]:

def CNNmodel(input_shape,filters=64, kernel=(3,3),size=9,dropout=0.5,**kwargs):
    _inputs = layers.Input(shape=input_shape)  # 48， 48， 1
    x=layers.Conv2D(32,kernel_size=[3,3],padding="same",activation=tf.nn.relu)(_inputs)
    x=layers.Conv2D(32, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)(x)
    x=layers.MaxPool2D(pool_size=[2,2],strides=2,padding="same")(x)
    x=layers.Conv2D(64,kernel_size=[3,3],padding="same",activation=tf.nn.relu)(_inputs)
    x=layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)(x)
    x=layers.MaxPool2D(pool_size=[2,2],strides=2,padding="same")(x)
    x=layers.Dropout(0.25,name='dropout1')(x)
    x=layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)(x)
    x=layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)(x)
    x=layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same")(x)
    x=layers.Dropout(0.25,name='dropout3')(x)
    x=layers.Conv2D(128, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)(x)
    x=layers.Conv2D(128, kernel_size=[1, 1], padding="same", activation=tf.nn.relu)(x)
    x=layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same")(x)
    x=layers.Dropout(0.25,name='dropout4')(x)
   
    
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(dropout, name='dropout')(x)  

    x=layers.Dense(128,activation=tf.nn.relu)(x)
    x = layers.Dense(7, activation='softmax', name='FC')(x)
    return keras.Model(inputs=_inputs,outputs=x)
def preprocess_input(inputs, std=255. ,mean=0., expand_dims=None):
    inputs = tf.cast(inputs,tf.float32)
    inputs = (inputs - mean) / std
    # print(inputs.shape)
    if expand_dims is not None:
        np.expand_dims(inputs,expand_dims)
    # print(inputs.shape)
    return inputs
def img_aug_fun(elem):
    elem = tf.image.random_flip_left_right(elem)#左右翻转
    elem = tf.image.random_brightness(elem, max_delta=0.5)#调亮度
    elem = tf.image.random_contrast(elem, lower=0.5, upper=1.5)#调对比度
    elem = preprocess_input(elem)
    return elem


train=tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\noname\Desktop\fer2013\Training", labels='inferred',label_mode='int',
    class_names=None, color_mode='grayscale', batch_size=32, image_size=(48,48), shuffle=True,interpolation='bilinear'
)
#test=train=tf.keras.preprocessing.image_dataset_from_directory(
  #  r"C:\Users\noname\Desktop\fer2013\PublicTest", labels='inferred',label_mode='int',
  #  class_names=None, color_mode='grayscale', batch_size=32, image_size=(48,48), shuffle=True,interpolation='bilinear'
#)



Found 28709 files belonging to 7 classes.


In [89]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train.map(lambda x, y: (normalization_layer(x), y))
normalized_ds = test.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [90]:
model = CNNmodel(input_shape=(48,48,1),filters=64, kernel=(3,3),size=9)
model.compile(optimizer='SGD',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train,validation_data=test,shuffle=True, batch_size=64, verbose=1,epochs=80)
model.save('my_model1.h5')

Epoch 1/80
898/898 [==============================] - 16s 17ms/step - loss: 1.9967 - accuracy: 0.2430 - val_loss: 1.7625 - val_accuracy: 0.2672
Epoch 2/80
898/898 [==============================] - 16s 17ms/step - loss: 1.7560 - accuracy: 0.2834 - val_loss: 1.7252 - val_accuracy: 0.3012
Epoch 3/80
898/898 [==============================] - 16s 17ms/step - loss: 1.7212 - accuracy: 0.3051 - val_loss: 1.7422 - val_accuracy: 0.2825
Epoch 4/80
898/898 [==============================] - 16s 18ms/step - loss: 1.6829 - accuracy: 0.3321 - val_loss: 1.6139 - val_accuracy: 0.3644
Epoch 5/80
898/898 [==============================] - 16s 18ms/step - loss: 1.6314 - accuracy: 0.3572 - val_loss: 1.7030 - val_accuracy: 0.3160
Epoch 6/80
898/898 [==============================] - 17s 18ms/step - loss: 1.5903 - accuracy: 0.3767 - val_loss: 1.5188 - val_accuracy: 0.4124
Epoch 7/80
898/898 [==============================] - 16s 18ms/step - loss: 1.5518 - accuracy: 0.3942 - val_loss: 1.6400 - val_accuracy:

In [91]:
# load model and test
model_restore = models.load_model('.\my_model1.h5')

OSError: SavedModel file does not exist at: .\my_model16.h5\{saved_model.pbtxt|saved_model.pb}

In [ ]:
print("Evaluate on test data")
privtest=tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\noname\Desktop\fer2013\PrivateTest", labels='inferred',label_mode='int',
    class_names=None, color_mode='grayscale', batch_size=32, image_size=(48,48), interpolation='bilinear'
)
results = model.evaluate(privtest, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data
Found 3589 files belonging to 7 classes.
113/113 [==============================] - 1s 11ms/step - loss: 1.1044 - accuracy: 0.5907
test loss, test acc: [1.1044200658798218, 0.5906937718391418]


  _inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32,(3,3),padding='same',use_bias=False,strides=(1,1), name='conv_0')(_inputs)
    x = layers.ReLU(6., name='conv_0_relu')(x)
    x = layers.Conv2D(64,(3,3),padding='same',use_bias=False,strides=(1,1), name='conv_1')(x)
    x = layers.ReLU(6., name='conv_1_relu')(x)
    x = layers.Conv2D(64,(3,3),padding='same',use_bias=False,strides=(1,1), name='conv_2')(x)
    x = layers.ReLU(6., name='conv_2_relu')(x)
   
    
    x = layers.Conv2D(64,(3,3),padding='same',use_bias=False,strides=(1,1), name='conv_3')(x)
    x = layers.ReLU(6., name='conv_3_relu')(x)
    

    x = layers.Conv2D(128,(3,3),padding='same',use_bias=False,strides=(1,1), name='conv_4')(x)
    x = layers.ReLU(6., name='conv_4_relu')(x)
